In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import tqdm as tqdm
import torch

In [3]:
import pandas as pd

# Load metadata
metadata = pd.read_csv("/projects/bdlo/music-sudden-cardiac-death/subject-info-cleaned.csv", index_col=0)

# Remove patients whose "Patient ID" is in the `erase` list
metadata = metadata[~metadata["Patient ID"].isin(erase)]  # Use .isin() and ~ for negation

# Save the filtered metadata back
metadata.to_csv("/projects/bdlo/music-sudden-cardiac-death/subject-info-cleaned.csv", index=True)

#print(f"Updated metadata saved. Remaining patients: {len(metadata)}")
metadata

,Patient ID,Follow-up period from enrollment (days),days_4years,Exit of the study,Cause of death,Age,Gender (male=1),Weight (kg),Height (cm),Body Mass Index (Kg/m2),...,Angiotensin-II receptor blocker (yes=1),Anticoagulants/antitrombotics (yes=1),Betablockers (yes=1),Digoxin (yes=1),Loop diuretics (yes=1),Spironolactone (yes=1),Statins (yes=1),Hidralazina (yes=1),ACE inhibitor (yes=1),Nitrovasodilator (yes=1)
1,P0002,2045,1460,0,0,58,1,74,160,289,...,1,1,1,0,0,0,1,0,0,0
2,P0004,2044,1460,0,0,56,0,84,165,309,...,1,1,1,0,1,1,0,0,0,0
3,P0006,2043,1460,0,0,70,1,83,165,305,...,0,1,1,0,1,1,0,0,1,1
4,P0007,2042,1460,0,0,52,1,71,162,271,...,0,1,1,0,0,0,1,0,0,0
5,P0009,2039,1460,0,0,64,0,68,155,283,...,0,1,1,0,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,P0907,771,771,3,6,50,1,69,169,242,...,0,1,1,0,1,0,1,1,0,1
601,P0913,463,463,3,6,69,0,70,151,307,...,0,1,1,0,1,0,0,0,1,0
602,P0978,608,608,3,6,69,1,106,170,367,...,0,1,1,0,1,0,1,0,1,0
603,P0981,214,214,3,6,73,1,63,160,246,...,1,1,1,0,1,0,0,0,0,1


In [5]:
#load ecg data
import wfdb
from tqdm import tqdm

# Specify the file path without the extension (it will look for .dat and .hea automatically)
ecg_dir = "/projects/bdlo/music-sudden-cardiac-death/Holter_ECG/"

erase = []
min_ = 3000000000000000000
for i in tqdm(metadata["Patient ID"].tolist()):
    name = ecg_dir + i
    # Load the signal and header info
    record = wfdb.rdrecord(name)  # Loads both .dat and .hea files
    signal = record.p_signal  # Get the ECG signal (numpy array)
    fields = record.__dict__  # Access additional metadata (like sampling frequency)
    if signal.shape[0] < min_:
        min_ = signal.shape[0]
        print(min_)




  0%|          | 1/583 [00:00<08:37,  1.12it/s]

16217800



  0%|          | 2/583 [00:01<08:42,  1.11it/s]

15539200



  1%|          | 3/583 [00:02<08:30,  1.14it/s]

...

  5%|▍         | 28/583 [00:28<09:51,  1.07s/it]

13847800


 37%|███▋      | 213/583 [03:43<06:14,  1.01s/it]

12609000


100%|██████████| 583/583 [10:11<00:00,  1.05s/it]


In [6]:
min_


12609000

In [17]:
#load ecg data
import wfdb

# Specify the file path without the extension (it will look for .dat and .hea automatically)
record_name = ecg_dir = "/projects/bdlo/music-sudden-cardiac-death/Holter_ECG/P0997"

# Load the signal and header info
record = wfdb.rdrecord(record_name)  # Loads both .dat and .hea files
signal = record.p_signal  # Get the ECG signal (numpy array)
fields = record.__dict__  # Access additional metadata (like sampling frequency)

print("Signal Shape:", signal.shape)

#trim
fs = fields['fs']
trim_seconds = 35
trim_samples = int(fs * trim_seconds)

signal = signal[trim_samples : -trim_samples]
print(signal.shape)
fields['sig_len'] = signal.shape

# Display basic info
print("Signal Shape:", signal.shape)
print("Sampling Frequency:", record.fs)
print("Channel Names:", record.sig_name)

Signal Shape: (17278400, 3)
(17264400, 3)
Signal Shape: (17264400, 3)
Sampling Frequency: 200
Channel Names: ['X', 'Y', 'Z']


In [6]:
len(record.sig_name)

2

In [1]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = "Data/subject-info-cleaned.csv"
df = pd.read_csv(file_path)

# Compute class distribution
class_counts = df["Cause of death"].value_counts()
min_class_size = class_counts.min()  # Smallest class size to downsample to

# Assign weights inversely proportional to class frequency
weights = df["Cause of death"].map(lambda x: 1 / class_counts[x])
weights /= weights.sum()  # Normalize weights

# Perform weighted downsampling to match the smallest class size
downsampled_df = df.sample(n=min_class_size * len(class_counts), weights=weights, random_state=42)

# Save the downsampled dataset
downsampled_df.to_csv("Data/subject-info-cleaned_downsampled.csv", index=False)

# Display the first few rows
print(downsampled_df.head())

     Unnamed: 0 Patient ID  Follow-up period from enrollment (days)  \
489         490      P0127                                      832   
594         595      P0872                                      166   
552         553      P0078                                      561   
528         529      P0838                                      335   
225         226      P0508                                     1855   

     days_4years  Exit of the study  Cause of death  Age  Gender (male=1)  \
489          832                  3               3   65                1   
594          166                  3               6   57                1   
552          561                  3               6   75                1   
528          335                  3               3   75                1   
225         1460                  0               0   66                1   

     Weight (kg)  Height (cm)  ...  Angiotensin-II receptor blocker (yes=1)  \
489           70          175  

In [2]:
# Load the downsampled dataset
downsampled_df = pd.read_csv("Data/subject-info-cleaned_downsampled.csv")

# Count number of instances for each class in "Cause of death"
downsampled_class_counts = downsampled_df["Cause of death"].value_counts()

# Display the results
print(downsampled_class_counts)

Cause of death
0    87
3    45
6    42
Name: count, dtype: int64


In [ ]:
#MYPORT=$(($(($RANDOM % 10000))+49152)); echo $MYPORT
#srun --account=bdlo-delta-cpu   --partition=cpu-interactive --time=00:30:00 --mem=32g jupyter-notebook --no-browser --port=<MYPORT> --ip=0.0.0.0
